In [1]:
import random
import math
import numpy as np
import cv2

import gym
from gym import Env, spaces

In [2]:
BLUE = [255, 0, 0]
#GREEN = [0, 255, 0]
RED = [0, 0, 255]

WHITE = [255, 255, 255]
GREY = [240, 240, 240]
#BLACK = [0, 0, 0]

In [48]:
# https://blog.paperspace.com/creating-custom-environments-openai-gym/
# https://towardsdatascience.com/creating-a-custom-openai-gym-environment-for-stock-trading-be532be3910e

class City(Env):
    def __init__(self):
        super(City, self).__init__()
        
        self.observation_shape = 5, 5
        self.observation_space = spaces.Box(low = 0, high = 1,
            shape = self.observation_shape, dtype = np.uint8)
        
        self.canvas_shape = 700, 700, 3 # width, height, color (BGR)
        self.canvas = np.ones(self.canvas_shape, dtype = np.uint8) * 0
        
        self.action_space = spaces.Tuple((
             spaces.Discrete(self.observation_shape[0] * self.observation_shape[1]),
             spaces.Discrete(self.observation_shape[0] * self.observation_shape[1])))
        #self.action_space = spaces.Discrete(self.observation_shape[0] * self.observation_shape[1],)
        
        self.map = np.ones(self.observation_shape, dtype = np.uint8) * 0
        self.offices = []
        self.houses = []
        pass
    
    def reset(self, random_start = False, start_shape = (4, 4)):
        self.map = np.ones(self.observation_shape, dtype = np.uint8) * 0
        self.offices = []
        self.houses = []
        
        if random_start :            
            for y in range((self.observation_shape[1] - start_shape[1]) // 2, (self.observation_shape[1] + start_shape[1]) // 2):
                for x in range((self.observation_shape[0] - start_shape[0]) // 2, (self.observation_shape[0] + start_shape[0]) // 2):
                    self.map[y, x] = random.randrange(3)
                    if self.map[y, x] == 1 : self.offices.append((x, y))
                    elif self.map[y, x] == 2 : self.houses.append((x, y))
        pass
    
    def __search_nearest_office(self, x, y):
        return int(min([math.dist((x, y), office) for office in self.offices]))
    
    def isFree(self,action_x,action_y):
        #on regarde si la case est ocuper
        error=False
        for i in self.houses:
            if i==(action_x, action_y):
                error=True
        for i in self.offices:
            if i==(action_x, action_y):
                error=True
        return error
    
    def step(self, action):
        
        action_xH, action_yH = action[0] % self.observation_shape[1], action[0] // self.observation_shape[1]
        errorHouse=self.isFree(action_xH, action_yH)
        
        self.houses.append((action_xH, action_yH))
        self.map[action_yH, action_xH] = 2
        self.draw_elements_on_canvas()
        
        
        action_x, action_y = action[1] % self.observation_shape[1], action[1] // self.observation_shape[1]
        
        errorOfficce=self.isFree(action_x, action_y)
        
        self.offices.append((action_x, action_y))
        self.map[action_y, action_x] = 1
        self.draw_elements_on_canvas()
        
        if errorHouse==True or errorOfficce==True:
            reward=-100
        else:
            reward = self.__search_nearest_office(action_xH, action_yH)
            reward = -reward + 5
        
        print(reward)
        
        return self.canvas, reward
    
    def __draw_element_on_canvas(self, x, y, color):
        observation_width, observation_height = self.observation_shape
        canvas_width, canvas_height, _ = self.canvas_shape

        drawing_width = int(canvas_width / observation_width)
        drawing_height = int(canvas_height / observation_height)

        # fit element to the canvas
        for j in range(y * drawing_height, y * drawing_height + drawing_height):
            for i in range(x * drawing_width, x * drawing_width + drawing_width):
                try : self.canvas[i, j] = color
                except IndexError : pass
                
        for j in range(y * drawing_height, y * drawing_height + drawing_height):
            try : self.canvas[x * drawing_width, j] = GREY
            except IndexError : pass
            
        for i in range(x * drawing_width, x * drawing_width + drawing_width):
            try : self.canvas[i, y * drawing_height] = GREY
            except IndexError : pass
        pass

    def draw_elements_on_canvas(self):
        
        # draw each element of the matrix
        for y in range(len(self.map)):
            for x in range(len(self.map[0])):
                color = WHITE
                if self.map[y, x] == 1 : color = BLUE # offices
                elif self.map[y, x] == 2 : color = RED # houses
                
                self.__draw_element_on_canvas(x, y, color)
        pass
    
    def render(self, mode = "human"):
        if mode == "human" :
            cv2.imshow("", self.canvas)
            cv2.waitKey(10)
        pass

In [49]:
#env = City()
#env.reset(random_start = True, start_shape = (4, 4))
#env.draw_elements_on_canvas()

#cv2.imshow("", env.canvas)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [50]:
env = City()
env.reset(random_start = True, start_shape = (4, 4))

for _ in range(10):
    env.step(env.action_space.sample())
    env.render()
    #faudrai aussi potenciellement suprimer les duplicata
    if (len(env.houses)+len(env.offices))>=(env.observation_shape[1]*env.observation_shape[1]):
        env.reset()
cv2.waitKey(0)
cv2.destroyAllWindows()

4
4
-100
-100
-100
4
-100
-100
-100
-100
